# Moodle 3

Con esta guía de Jupyter vamos a lograr instalar una versión reciente de Moodle, en un servidor Ubuntu 16.04 recién instalado.

Primero debe instalar los componentes del servidor "LEMP", inciciando por la instalación de la base de datos. Seguidamente descargamos el código de Moodle.

Por último, creamos la base de datos a ser utilizada por Moodle, y luego corremos el instalador Web.


### PostgreSQL

PostgreSQL es motor de base de datos relacional con una impresionante cantidad de características como pueden ser manejo de cubos, vistas, procedimientos almacenados, GIS, replicación de multi-maestro entre otras.

Se recomienda por parte de varios administradores de Drupal por asuntos de desempeño, por lo que en esta guía veremos este poderoso motor.

In [ ]:
sudo apt-get install -y postgresql postgresql-contrib

### Nginx y Php

Procedemos ahora a instalar Nginx y Php en versión 'FPM'.

Así mismo instalamos módulos de Php requeridos por la aplicación Moodle, según la documentación del fabricante.

Nota: No vamos a reiniciar el servicio de Nginx en este momento, sino hasta luego de editar archivos de configuración.

In [ ]:
sudo apt-get -y install php-fpm php-curl php-gd php-xmlrpc php-intl php-xml php-zip \
 php-mbstring php-soap php-pgsql
sudo apt-get -y install nginx

### Cambios menores en Apache

**Desde una terminal**, y utilizando su editor preferido, debe cambiar dos archivos de la siguiente forma:

**Archivo:** ``/etc/nginx/sites-enabled/default``

**Cambio:** Vamos eliminar las líneas existentes y a agregar esta definición de virtualhost.

```
server {
    server_name moodle.ejemplo.com;
    listen 80;
        root /var/www/html;
        rewrite ^/(.*\.php)(/)(.*)$ /$1?file=/$3 last;

        location ^~ / {
                try_files $uri $uri/ /index.php?q=$request_uri;
                index index.php index.html index.htm;

                location ~ \.php$ {
                        include snippets/fastcgi-php.conf;
                        fastcgi_pass unix:/run/php/php7.0-fpm.sock;
                }
        }
}   
```

Reiniciamos Nginx y mostramos el estado del servicio.

## Moodle

La última versión y recomendada de Moodle es la 3.3.1 que vamos a descargar. Sin embargo lo vamos a hacer vía **Git** que es la forma preferida para algunos administradores de Moodle, por lo que vamos a definir el repo en el *Branch* llamado **MOODLE_33_STABLE**.

**Importante**: El repositorio pesa varios cientos de Megas (~, dado que no es solo el código de Moodle sino también la historia del repositorio. Debe evaluar esto como una ventaja/desventaja vs los métodos utilizados en el tutorial de Apache, y recordar que el método de Git es completamente independiente a Apache/Nginx. En ambiente de aula se recomienda realizar la descarga de forma local, y copiarla, para no descargar varias veces el repositorio.

Vamos a instalar la herramienta de Git así como la herramienta de Acls en Linux, para darnos permisos de escritura al directorio.

In [ ]:
sudo systemctl restart nginx
sudo systemctl status nginx | cat
sudo apt-get install -y git acl
git clone https://github.com/moodle/moodle.git
cd moodle
git remote show origin
git checkout MOODLE_33_STABLE
ls -lh

Descomprimimos el código de Moodle dentro de /var/www/html que es el directorio default de Nginx en Debian/Ubuntu.

En un sistema en producción se recomienda crear un directorio para cada virtualhost, o un único virtualhost para Moodle y hacer uso de la función de "Multi tenant" para trabajar varios clientes con una sola instancia de la aplicación.

Importante que el código queda con derechos de **www-data**, usuario que utiliza Debian para el servicio de web de Nginx. Esto porque Moodle debe poder manipular archivos dentro de la carpeta.

In [ ]:
cd
sudo rm -r /var/www/html
sudo mv moodle /var/www/html
sudo chown -R www-data:www-data /var/www/html
sudo setfacl -R -m u:greencore:rwX /var/www/html
ls /var/www/html

### Creación de base de datos

Antes de ejecutar el instalador Web de Moodle debemos crear la base de datos y el usuario que usará la aplicación de Moodle.

Para ello vamos a utilizar el comando de mysql(1), definiendo la contraseña "greencore" en la misma línea de comando.

In [ ]:
sudo -u postgres psql << EOF
CREATE USER moodle WITH PASSWORD 'greencore';
CREATE DATABASE moodle;
GRANT ALL PRIVILEGES ON DATABASE moodle to moodle;
\q
EOF

## Configuración de Moodle

Para el caso de Moodle, hace falta un directorio adicional llamado ``/var/www/moodledata`` el cual debemos crear y asignar permisos para que Moodle pueda crear contenido en este lugar cuando sea necesario.

In [ ]:
sudo mkdir /var/www/moodledata
sudo chown -R www-data /var/www/moodledata
sudo chmod -R 0770 /var/www/moodledata

### Instalador web

No es necesario manipular ningún archivo Php para definir la configuración de nuestro sitio. Visitamos el sitio web, donde nos recibe el instalador Web de Moodle. Este funciona en estilo "wizard" donde simplemente vamos contestando con cuidado las consultas que nos hace.

http://10.42.25.6X/

**Importante**: Seleccione el "Database driver" de **PostgreSQL** en lugar de Improved Mysql.

En el instalador puede usar valores que usaría en un sitio en producción.
Recuerde utilizar para este laboratorio el usuario "admin", aunque no va a aceptar la contraseña "greencore" para facilitar la revisión de errores o dudas. Se recomienda **Gr33nc0r3.** para este laboratorio.